# Demo for using `hifir4py` with user `Params` and  SciPy sparse matrix#
In this example, we show how to use `hifir4py` HIFIR preconditioner coupling with the built-in GMRES solver.. The example system is a saddle-point formulation of 3D Stokes equation with Taylor-Hood elements.

In [1]:
from scipy.io import loadmat
from hifir4py import *
import numpy as np

In [2]:
# load the MATFILE from scipy.io
f = loadmat("demo_inputs/matlab.mat")
A = f["A"]
b = f["b"].reshape(-1)

Let's show some basic information of the system, including shape, nnz, and leading block symmetry

In [3]:
# A is scipy.sparse.csr_matrix
print('The system shape is {}, where the nnz is {}'.format(A.shape, A.nnz))

The system shape is (2990, 2990), where the nnz is 44632


Now, let's build the preconditioenr $\boldsymbol{M}$ with more aggressive options, i.e. `droptol` for L and U factors is 1e-2, `condest` for L, U, and D is 5, and $\alpha$ for L and U is 3.

In [4]:
M = HIF()
params = Params()
params['tau_L'] = params['tau_U'] = 0.01
params['kappa'] = params['kappa_d'] = 5.0
params['alpha_L'] = params['alpha_U'] = 3
M.factorize(A, params=params)


|           Hybrid (Hierarchical) Incomplete Factorizations           |
|                                                                     |
| HIF is a package for computing hybrid (hierarchical) incomplete fa- |
| ctorizations with nearly linear time complexity.                    |
-----------------------------------------------------------------------

 > Package information:

		* Copyright (C) The HIF AUTHORS
		* Version: 1.0.0
		* Built on: 11:33:49, Jun  7 2021


Params (control parameters) are:

tau_L                         0.010000
tau_U                         0.010000
kappa_d                       5.000000
kappa                         5.000000
alpha_L                       3.000000
alpha_U                       3.000000
rho                           0.500000
c_d                           10.000000
c_h                           2.000000
N                             -1
verbose                       info
rf_par                        1
reorder                       Auto
s

With the preconditioenr successfully been built, let's print out some basic information

In [5]:
print('M levels are {}, with nnz {}'.format(M.levels, M.nnz))

M levels are 2, with nnz 90664


Now, we solve with the built-in flexible GMRES solver, with default configurations, i.e. restart is 30, relative convergence tolerance is 1e-6, and maximum allowed iterations are 500.

In [6]:
solver = GMRES(M)

In [7]:
x, iters = solver.solve(A, b)

- GMRES -
rtol=1e-06
restart/cycle=30
maxiter=500
flex-kernel: tradition
init-guess: no

Calling traditional GMRES kernel...
Enter outer iteration 1...
  At iteration 1, relative residual is 0.000216346.
  At iteration 2, relative residual is 5.54575e-06.
  At iteration 3, relative residual is 4.62893e-07.


In [8]:
print('solver done, with {} iterations and residule is {}'.format(iters, solver.resids[-1]))

solver done, with 3 iterations and residule is 5.405694712085073e-06
